<a href="https://colab.research.google.com/github/ankuu63/TextpredictorandGenerator/blob/main/Text_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Text genrator using LSTM**


1. Dataset from kaggle
2. Downloading with help of datasets
3. using kaggle_datset_link, username , kagle apitoken_key

In [2]:
! pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ashishpandey2062/next-word-predictor-text-generator-dataset")


Skipping, found downloaded files in "./next-word-predictor-text-generator-dataset" (use force=True to force download)


Reading dataset

In [4]:
with open("/content/next-word-predictor-text-generator-dataset/next_word_predictor.txt", "r", encoding="utf-8") as file:
    data = file.read()

Data cleaning of extra things like #@$%^&* type symbols and punctuation
using **Separate Punctuation**  function

In [5]:
def separate_punc(doc_text):

    return [token.lower() for token in doc_text.split(" ") if token not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [6]:
data= separate_punc(data)
df= " ".join(data)


*_**Importing libraries**_*

In [7]:
import tensorflow as tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense ,  Dropout, BatchNormalization
import numpy as ny
from keras.regularizers import l2
import time
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")



Tokenization of datasets to form vocab **Dictionary**


In [8]:
#tokenizer as object for Teokenizer class
tokenizer= Tokenizer(num_words=None,char_level=False)
tokenizer.fit_on_texts([df])

#tokens for unique words in df.
vocab= tokenizer.word_index
print(vocab)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'i': 6, 'you': 7, 'in': 8, 'is': 9, 'monica': 10, 'it': 11, 'with': 12, 'ross': 13, 'that': 14, 'rachel': 15, 'for': 16, 'chandler': 17, 'this': 18, 'on': 19, 'joey': 20, 'was': 21, 'oh': 22, 'phoebe': 23, 'are': 24, 'all': 25, 'as': 26, 'what': 27, 'be': 28, 'like': 29, 'no': 30, "it's": 31, "i'm": 32, 'her': 33, 'they': 34, 'just': 35, 'from': 36, 'okay': 37, 'not': 38, 'so': 39, 'my': 40, 'have': 41, 'me': 42, 'where': 43, 'know': 44, 'she': 45, 'we': 46, 'out': 47, 'well': 48, 'their': 49, 'can': 50, 'at': 51, 'he': 52, 'yeah': 53, 'your': 54, 'about': 55, 'but': 56, 'its': 57, 'up': 58, "don't": 59, 'text': 60, 'scene': 61, 'by': 62, 'do': 63, 'an': 64, 'or': 65, 'were': 66, 'there': 67, 'if': 68, 'uh': 69, 'look': 70, 'life': 71, 'through': 72, 'into': 73, 'him': 74, 'his': 75, "you're": 76, 'hey': 77, 'how': 78, 'right': 79, 'think': 80, 'time': 81, 'now': 82, 'paul': 83, 'people': 84, 'had': 85, 'world': 86, 'who': 87, "that's": 88

vocab size " vocab_size"  Here size is 4993

In [9]:

vocabsize= len(vocab)
print(vocabsize)

4993


1.Splitting data into sentences on basis of tab seperation


2.Generating InputSequences coresponding to each sentences

In [10]:
input_seq=  []

#splitting sentences
for sentences in df.split("\n"):
   tokenized_sentences= tokenizer.texts_to_sequences([sentences])[0]             #tokenizng_senteces
          #storing tokenized sentences into a inputseq list
   for i in range(1,len(tokenized_sentences)):
    input_seq.append(tokenized_sentences[:i+1])




print(input_seq[: 20])



[[1, 155], [1, 155, 21], [1, 155, 21, 2368], [1, 155, 21, 2368, 1549], [1, 155, 21, 2368, 1549, 8], [1, 155, 21, 2368, 1549, 8, 1], [1, 155, 21, 2368, 1549, 8, 1, 422], [1, 155, 21, 2368, 1549, 8, 1, 422, 692], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550, 2370], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550, 2370, 1], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550, 2370, 1, 423], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550, 2370, 1, 423, 4], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550, 2370, 1, 423, 4, 1], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2, 3, 2369, 1550, 2370, 1, 423, 4, 1, 1142], [1, 155, 21, 2368, 1549, 8, 1, 42

**Zeropadding**


Since every sentence has diff len thus their token has too : making them of even len by zero padding


In [11]:
# Finding max length of tokeinzed sentence
max_len = max([len(x) for x in input_seq])
print(max_len)

#zeropadding
padded_input_seq= pad_sequences(input_seq, maxlen=max_len, padding ='pre')
print(padded_input_seq[:10])

325
[[   0    0    0 ...    0    1  155]
 [   0    0    0 ...    1  155   21]
 [   0    0    0 ...  155   21 2368]
 ...
 [   0    0    0 ...    1  422  692]
 [   0    0    0 ...  422  692  215]
 [   0    0    0 ...  692  215    2]]


Splitting the Padded input sequences into **X: Training seq , Y: Corresponding output(from: discreate number)**

In [12]:
x = padded_input_seq[:, :-1]
y = padded_input_seq[: ,-1]

#####coresponding shape of X and Y


print(x.shape)

print(y.shape)

(26383, 324)
(26383,)


Now y has to be **one hot encoded** so that disreate-> each array row of length = len of vocab

In [13]:
y= to_categorical(y, num_classes= vocabsize +1) ##since one hot encoding start ferom 0 but token from 1
                                              ##so to takelast token into one hot it must be of vocablen+1
print(y.shape)


(26383, 4994)


**LSTM Model_architecture**  **Training**

In [14]:
model=Sequential()
model.add(Embedding(4994, 100, input_length=324))
model.add(LSTM(128))
model.add(Dense(4994, activation='softmax'))

model.compile(loss="binary_crossentropy", optimizer= "adam", metrics=["accuracy"] )
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(x,y,epochs=150)

Epoch 1/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.0404 - loss: 0.0797
Epoch 2/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.0468 - loss: 0.0017
Epoch 3/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0521 - loss: 0.0016
Epoch 4/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0486 - loss: 0.0016
Epoch 5/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0498 - loss: 0.0016
Epoch 6/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0486 - loss: 0.0016
Epoch 7/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.0497 - loss: 0.0016
Epoch 8/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.0502 - loss: 0.0016
Epoch 9/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0505 - loss: 0.0016
Epoch 10/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0492 - loss: 0.0016
Epoch 11/150
825/825 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0497 - loss: 0.0016
Epoch 12/150
825/82

 **TESTING : Next Word predictor by LSTM**

In [17]:
text=" please let me know "


for i in range(15):

    # Tokenize the input text
    token_text = tokenizer.texts_to_sequences([text])[0]

    # Pad the tokenized text
    padded_token_text = pad_sequences([token_text], maxlen=324, padding='pre')

    # Predict the next word index
    pos = ny.argmax(model.predict(padded_token_text))

    # Retrieve the word corresponding to the predicted index
    for word, index in tokenizer.word_index.items():
        if index == pos:
            # Append the predicted word to the input text
            text = text + " " + word
            print(text)


            time.sleep(1) ##delay 1 sec

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
 please let me know  what
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
 please let me know  what if
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
 please let me know  what if you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
 please let me know  what if you let
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
 please let me know  what if you let them
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
 please let me know  what if you let them it
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
 please let me know  what if you let them it was
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
 please let me know  what if you let them it was like
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
 please let me know  what if you let them it was like when
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
 please let me know  what if you let them it was like when you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
 please let me know  what if you let them it was like when you can
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
 please let me know  what if you let the